In [ ]:
!pip install -U sentence-transformers
!pip install feedparser requests scikit-learn googletrans==4.0.0-rc1 sentence-transformers
!pip install newsapi
!pip install newsapi-python
!pip install --upgrade transformers
!pip install beautifulsoup4
!pip install requests_html
!pip install gradio


  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.27.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio 4.27.0 requires urllib3~=2.0, but you have urllib3 1.26.18 which is incompatible.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: urllib3
 

In [ ]:
!pip install --upgrade googletrans==4.0.0-rc1


  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.5
    Uninstalling httpcore-1.0.5:
      Successfully uninstalled httpcore-1.0.5
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.27.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio-client 0.15.1 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.


In [ ]:

import requests
from sentence_transformers import SentenceTransformer, util
from googletrans import Translator
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from transformers import pipeline

def extract_first_5_paragraphs(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')[:5]  # Get the first 5 <p> tags
            concatenated_paragraph = ' '.join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])
            return concatenated_paragraph  # Return the concatenated paragraph
        else:
            print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

In [ ]:
def extract_semantic_paragraphs(url,source_language):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')[:5]  # Get the first 5 <p> tags
            sum=0
            for p in paragraphs:
              text = p.get_text()
              translated_text = translate_to_english(text, source_language)
              sentence_similarity = calculate_similarity_score(query,translated_text)
              sum+=sentence_similarity
              return sum
        else:
            print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

In [ ]:
def extract_entire_paragraphs(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')[:6]

            # Concatenate text content of all <p> tags
            concatenated_paragraph = ""
            for paragraph in paragraphs:
                paragraph_text = paragraph.get_text(strip=True)
                if paragraph_text:  # Check if paragraph text is not empty
                    concatenated_paragraph += paragraph_text + " "

            return concatenated_paragraph.strip()  # Return the concatenated paragraph, stripped of extra spaces

        else:
            print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
            return None

    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

In [ ]:
def translate_to_english(text, source_language):
    """
    Translate text to English using Google Translate API.

    Args:
    - text (str): The text to be translated.
    - source_language (str): The source language code of the text (e.g., 'kn' for Kannada, 'ta' for Tamil).

    Returns:
    - translated_text (str): The translated text in English.
    """
    translator = Translator()
    translated = translator.translate(text, src=source_language, dest='en')
    translated_text = translated.text
    return translated_text

In [ ]:
def classify_query(query):
    classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
    hypothesis_template = 'This text is about {}.'
    labels = [
        "Politics", "Business", "Technology", "Sports", "Entertainment", "Health",
        "Science", "Environment", "Education", "Finance", "Crime", "Lifestyle",
        "Travel", "Weather", "Culture", "Religion", "Food", "Fashion", "Arts", "Opinions"
    ]

    prediction = classifier(query, labels, hypothesis_template=hypothesis_template, multi_class=True)
    predicted_label = prediction['labels'][0]  # Assuming the highest scored label is the predicted category
    print(predicted_label)
    return predicted_label

In [ ]:
def calculate_similarity_score(query, title):
    """
    Calculate the semantic similarity score between the query and article title.

    Args:
    - query (str): The search query.
    - title (str): The title of the article.

    Returns:
    - similarity_score (float): The semantic similarity score.
    """
    # Use SentenceTransformer to calculate semantic similarity
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query)  # Encode the query
    title_embedding = model.encode(title)  # Encode the article title
    similarity_score = util.pytorch_cos_sim(query_embedding, title_embedding).item()
    return similarity_score


In [ ]:

from transformers import pipeline

from transformers import pipeline

def summarisation(text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summarised_text = summarizer(text, max_length=400, min_length=30, do_sample=False)
    summary = summarised_text[0]['summary_text']
    sentences = summary.split('.')
    formatted_summary = '\n'.join([sentence.strip() for sentence in sentences if sentence.strip()])

    return formatted_summary


In [ ]:
def get_articles(query, page_size=10):
    """
    Retrieve relevant articles from French and Spanish news sources based on a query using News API.

    Args:
    - query (str): The search query in English.
    - page_size (int): The maximum number of articles to return (default is 10).

    Returns:
    - articles (list): A list of dictionaries, each containing information about a relevant article.
    """
    category = classify_query(query)

    # Define News API endpoint and API key
    api_key = 'fd8d3b8bf4654a15b631eea5066bc001'
    news_api_endpoint = 'https://newsapi.org/v2/everything'

    # Set languages to French and Spanish
    languages = ['es','fr','en']

    # Calculate the date range for the past 24 hours
    end_date = datetime.now()
    start_date = end_date - timedelta(days=3)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Initialize list to store all relevant articles
    relevant_articles = []

    for language in languages:
        # Define query parameters for each language
        params = {
            'q': category,
            'pageSize': page_size,
            'language': language,
            'apiKey': api_key,
            'from': start_date_str,
            'to': end_date_str
        }

        # Make request to News API
        response = requests.get(news_api_endpoint, params=params)
        articles_data = response.json()

        # Parse articles data for the current language
        for article in articles_data['articles']:
            title = article['title']
            url = article['url']
            description = article['description']

            # Translate title to English for similarity score calculation
            translated_title = translate_to_english(title, language)

            # Calculate similarity score between query and translated title
            similarity_score = calculate_similarity_score(query, translated_title)

            # Filter articles based on similarity score threshold
            if similarity_score > 0.07:  # Adjust the threshold as needed
                article_info = {
                    'title': translated_title,
                    'language': language,  # Store the language of the article
                    'similarity_score': similarity_score,
                    'url': url,
                    'description': description
                }
                relevant_articles.append(article_info)

    # Sort articles by similarity score (descending)
    relevant_articles.sort(key=lambda x: x['similarity_score'], reverse=True)

    # Limit to top relevant articles
    top_relevant_articles = relevant_articles[:20]  # Adjust the number of top articles as needed

    # Process top relevant articles
    for article in top_relevant_articles:
        content = article['description']
        # Translate sentences to English
        if content == "Comments":
            article['translated_desc'] = "None"
            continue
        else:
            translated_desc = translate_to_english(content, 'auto')
            content_similarity_scores = calculate_similarity_score(query, translated_desc)
            if content_similarity_scores is not None:
                if float(content_similarity_scores) > float(article['similarity_score']):
                    article['similarity_score'] = content_similarity_scores
            article['translated_desc'] = translated_desc

    # Sort top relevant articles based on the highest similarity scores
    top_relevant_articles.sort(key=lambda x: x['similarity_score'], reverse=True)

    # Get the top 5 relevant articles
    top_10_relevant_articles = top_relevant_articles[:10]

    # Retrieve additional data (e.g., semantic scores, images) for the top 5 relevant articles
    for article in top_10_relevant_articles:
        article_url = article['url']
        semantic_score = extract_semantic_paragraphs(article_url, article['language'])
        if semantic_score is not None:
            article['semantic_score'] = semantic_score
        else:
            article['semantic_score'] = article['similarity_score']
        concatenated_paragraph = extract_first_5_paragraphs(article_url)
        if concatenated_paragraph and concatenated_paragraph.strip():
            translated_paragraph = translate_to_english(concatenated_paragraph, article['language'])
            para_similarity_score = calculate_similarity_score(query, translated_paragraph)
            article['para_sim_score'] = para_similarity_score
            translated_sentences = translated_paragraph.split('.')
            translated_text = '\n'.join([sentence.strip() for sentence in translated_sentences if sentence.strip()])
            article['actual_content'] = translated_text
        else:
            article['para_sim_score'] = None
            print("No paragraphs extracted from the webpage.")

    # Assuming top_10_relevant_articles is your list of dictionaries containing articles

    top_10_relevant_articles.sort(key=lambda x: x['semantic_score'] if x['semantic_score'] is not None else 0, reverse=True)


     #Get the top 5 relevant articles y
    top_5_relevant_articles = top_10_relevant_articles[:5]
    best_article = top_5_relevant_articles[0]
    entire_paragraph = extract_entire_paragraphs(best_article['url'])

    if entire_paragraph is not None:
      translated_content = translate_to_english(entire_paragraph, best_article['language'])
      summarised_content = summarisation(translated_content)
      best_article['Entire_content']= summarised_content
    else:
      best_article['Entire_content']= translated_paragraph


    for article in top_5_relevant_articles:
        article_url = article['url']
        try:
            response = requests.get(article_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            images = soup.find_all('img')
            captions = [img['alt'] if 'alt' in img.attrs else None for img in images]
            images_with_captions = [{'url': img['src'], 'caption': translate_to_english(caption, 'auto')} for img, caption in zip(images, captions) if caption]

            # Calculate similarity between query and image captions
            similarity_scores = [calculate_similarity_score(query, img_info['caption']) for img_info in images_with_captions]

            # Get the indices of the 5 most relevant images
            top_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:5]

            # Add the 5 most relevant images to the article
            article['images'] = [images_with_captions[i] for i in top_indices]

        except Exception as e:
            print(f"Error retrieving images for article {article_url}: {e}")
            article['images'] = []


    return best_article

In [ ]:
query = 'whats going on in germany'
articles = get_articles(query)

# print(f"Article {idx}:")
print("Title:", articles['title'])
print("Language:", articles['language'])
print("Similarity Score:", articles['similarity_score'])
print("para sim score:", articles['para_sim_score'])
print("semantic para score:", articles['semantic_score'])
print("Description:", articles['translated_desc'])
# print("Content:", article['actual_content'])
print("Summarised Content:", articles['Entire_content'])
print("URL:", articles['url'])

# Display the first image that does not encounter an error
if articles['images']:
    print("Images:")
    for img_info in articles['images']:
        print("- Caption:", img_info['caption'])
        try:
            response = requests.get(img_info['url'])
            img_data = BytesIO(response.content)
            img_obj = Image.open(img_data)
            plt.figure()
            plt.imshow(img_obj)
            plt.axis('off')
            plt.show()
            break  # Stop after displaying the first image
        except Exception as e:
            print(f"Error displaying image: {e}")
    else:
      print("No images found for this article.")
else:
  print("Thank you!")

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


Culture
Error fetching content from https://www.businessinsider.com/russias-military-is-allergic-to-the-deep-reforms-it-needs-2024-4: the JSON object must be str, bytes or bytearray, not NoneType
No paragraphs extracted from the webpage.
Failed to fetch content from https://bitmovin.com/careers. Status code: 403
Failed to fetch content from https://bitmovin.com/careers. Status code: 403
No paragraphs extracted from the webpage.


TypeError: the JSON object must be str, bytes or bytearray, not NoneType